In [5]:
# 아이템기반 최근접 이웃 협업 필터링 구현 예제
import pandas as pd
# 데이터 불러오기
movies = pd.read_csv('./data/movies.csv') # 책에서 사용했던 movies.csv
ratings = pd.read_csv('./data/ratings.csv') # 책에서 사용했던 ratings.csv

In [6]:
ratings = ratings[['userId','movieId','rating']]

#1. ratings 데이터와 movies 데이터 movieId 기준으로 병합하기
ratings_movies = pd.merge(ratings, movies, on='movieId')

In [7]:
#2. 값은 rating, 행은 userId, 열은 title 인 pivot_table 만들기
ratings_matrix = ratings_movies.pivot_table('rating', index='userId', columns='title')
# 결측치 처리
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# 3. 영화 간 유사도 추출을 위해 ratings_matrix 전치하기
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 4. 영화간 코사인 유사도 구하기

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns = ratings_matrix.columns)
item_sim_df.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [14]:
# 5. Toy Story (1995) 와 유사한 영화 10개 확인 (자기자신 제외)
item_sim_df['Toy Story (1995)'].sort_values(ascending=False)[1:11]

title
Toy Story 2 (1999)                                   0.572601
Jurassic Park (1993)                                 0.565637
Independence Day (a.k.a. ID4) (1996)                 0.564262
Star Wars: Episode IV - A New Hope (1977)            0.557388
Forrest Gump (1994)                                  0.547096
Lion King, The (1994)                                0.541145
Star Wars: Episode VI - Return of the Jedi (1983)    0.541089
Mission: Impossible (1996)                           0.538913
Groundhog Day (1993)                                 0.534169
Back to the Future (1985)                            0.530381
Name: Toy Story (1995), dtype: float64

#### 구남이

In [15]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

# 데이터 로딩
anime =pd.read_csv('./data/anime.csv')
print(anime.shape)
anime.head(1)

(17562, 35)


,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0


In [16]:
# 1. anime 데이터 프레임에서 
# 'MAL_ID','Name', 'Score', 'Genres', 'Studios' 열만 추출해서 anime_df에 저장
anime_df = anime[['MAL_ID','Name', 'Score', 'Genres', 'Studios' ]]

In [18]:
# 2. CountVectorizer로 피처 벡터화하기
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(anime_df['Genres'])

In [19]:
# 3. 생성된 피처 벡터 행렬의 코사인 유사도 계산하기
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)

In [20]:
# 4. 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값을
# genre_sim_sorted_ind 에 저장
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]

def find_sim_movie(df, sorted_ind, name, top_n=10):
 
    name_anime = df[df['Name'] == name]

    index = name_anime.index.values
    similar_indexes = sorted_ind[index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [21]:
# 카우보이 비밥과 장르별로 유사한 애니메이션 10개 추천
similar_anime = find_sim_movie(anime_df, genre_sim_sorted_ind, 'Cowboy Bebop',10)
similar_anime['Name']

[[   0 3454  927 2205 3502 2016 1352 2015 1079 1064]]


0                                            Cowboy Bebop
3454                      Cowboy Bebop: Yose Atsume Blues
927                                        Generator Gawl
2205    Keroro Gunsou Movie 2: Shinkai no Princess de ...
3502                              Trigun: Badlands Rumble
2016             Waga Seishun no Arcadia: Mugen Kidou SSX
1352                            Ginga Tetsudou Monogatari
2015                              Waga Seishun no Arcadia
1079                                        Lost Universe
1064                          Musekinin Kanchou Tylor OVA
Name: Name, dtype: object

#### 강수민

In [22]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, 6, np.NaN, 2, np.NaN ],
              [np.NaN, np.NaN, 3, np.NaN, 1],
              [4, np.NaN, np.NaN, 3,  4 ],
              [np.NaN, 2, 1, 1, np.NaN ]])
num_users, num_items = R.shape
K=3

In [23]:
# Q1. P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다. 
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users,K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [24]:
# 오차 구하기
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [25]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장.(널 값을 제외한 데이터의 행렬 인덱스) 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. (steps는 반복해서 업데이트 할 횟수)
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Q2. Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.1310205474303294
### iteration step :  50  rmse :  0.408831934116086
### iteration step :  100  rmse :  0.10216788232410844
### iteration step :  150  rmse :  0.03549077318020086
### iteration step :  200  rmse :  0.01801245040558789
### iteration step :  250  rmse :  0.01401813749569551
### iteration step :  300  rmse :  0.013052196928678336
### iteration step :  350  rmse :  0.012767705472425766
### iteration step :  400  rmse :  0.012671139542813816
### iteration step :  450  rmse :  0.012636233484464084
### iteration step :  500  rmse :  0.012623949281226109
### iteration step :  550  rmse :  0.012620631230271715
### iteration step :  600  rmse :  0.012621064961960282
### iteration step :  650  rmse :  0.01262311406628527
### iteration step :  700  rmse :  0.01262585151438593
### iteration step :  750  rmse :  0.012628857091456534
### iteration step :  800  rmse :  0.01263193476066944
### iteration step :  850  rmse :  0.012634991967094897
### ite

In [26]:
    # Q3. 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
pred_matrix =  np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[ 3.99   5.979  1.059  2.005  4.172]
 [ 0.124  2.979  2.982  0.165  0.994]
 [ 3.992  3.742 -0.996  2.982  3.985]
 [ 0.954  1.996  0.994  0.987  1.405]]


In [27]:
R

array([[ 4.,  6., nan,  2., nan],
       [nan, nan,  3., nan,  1.],
       [ 4., nan, nan,  3.,  4.],
       [nan,  2.,  1.,  1., nan]])